# Exercises

- You can assume that the datatype of all tensors is **FP16**.
## Section 1. Wave Quantization

### Problem 1-1
We will apply 2D convolution by this settings.
- Batch size: 20
- Input tensor: height = width = 15, channel = 64
- Filter: 8x8 size
- Output tensor: channel = 3072
- Tensor layout: NHWC
- Tile size: 256x128

**Task:** Find **two output channel values** which don't cause wave quantization. The values should be in range `[2000, 5000]`.


In [ ]:
# Code for problem 1-1
import tensorflow as tf
from tensorflow import nn as nn
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# diltion, padding, and stride are not accounted for!
# They are assumed to be default values
def get_conv2d_MNK(in_tensor, filter, layout):
    if layout == "NHWC":
        batch_size, in_height, in_width, in_channels = in_tensor.shape

    if layout == "NCHW":
        batch_size, in_channels, in_height, in_width = in_tensor.shape

    filter_height, filter_width, in_channels, out_channels = filter.shape

    out_height = in_height - (filter_height - 1)
    out_width = in_width - (filter_width - 1)

    M = batch_size * out_height * out_width
    N = out_channels
    K = in_channels * filter_width * filter_height

    return (M, N, K)


def approximate_gflops(in_tensor, filters, layout, time):
    if time <= 0:
        raise ValueError("time must be greater than 0")
    M, N, K = get_conv2d_MNK(in_tensor, filters, layout)
    flop = 2*M*N*K
    flops = flop / time
    gflops = flops / 1000000000
    return gflops


### DO NOT CHANGE THESE VALUES ###
batch_size = 20
in_height = 15
in_width = 15
in_channels = 64
filter_height = 8
filter_width = 8
layout = "NHWC"

### CHANGE HERE! ###
out_channels = 3072

### DO NOT CHANGE BELOW ###
in_tensor = tf.random.uniform([batch_size, in_height, in_width, in_channels], dtype=tf.dtypes.float16)
filters = tf.random.uniform([filter_height, filter_width, in_channels, out_channels], dtype=tf.dtypes.float16)

# Warming up
%timeit -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)

# Profile!
elapsed_time = %timeit -o -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)
print("out_channels %d: %f GFLOPS" % (out_channels, approximate_gflops(in_tensor, filters, layout, elapsed_time.average)))


### Problem 1-2
We will apply 2D convolution by this settings.
- Batch size: 56
- Input tensor: height = width = 15, channel = 64
- Filter: 8x8 size
- Output tensor: channel = 2560
- Layout: NHWC
- Tile size: 256x64

**Task:** Find **one batch size value** which doesn't cause wave quantization. The value should be in range `[40, 60]`.

In [ ]:
# Code for problem 1-2
import tensorflow as tf
from tensorflow import nn as nn
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# diltion, padding, and stride are not accounted for!
# They are assumed to be default values
def get_conv2d_MNK(in_tensor, filter, layout):
    if layout == "NHWC":
        batch_size, in_height, in_width, in_channels = in_tensor.shape

    if layout == "NCHW":
        batch_size, in_channels, in_height, in_width = in_tensor.shape

    filter_height, filter_width, in_channels, out_channels = filter.shape

    out_height = in_height - (filter_height - 1)
    out_width = in_width - (filter_width - 1)

    M = batch_size * out_height * out_width
    N = out_channels
    K = in_channels * filter_width * filter_height

    return (M, N, K)


def approximate_gflops(in_tensor, filters, layout, time):
    if time <= 0:
        raise ValueError("time must be greater than 0")
    M, N, K = get_conv2d_MNK(in_tensor, filters, layout)
    flop = 2*M*N*K
    flops = flop / time
    gflops = flops / 1000000000
    return gflops


### DO NOT CHANGE THESE VALUES ###
in_height = 15
in_width = 15
in_channels = 64
filter_height = 8
filter_width = 8
out_channels = 2560
layout = "NHWC"

### CHANGE HERE! ###
batch_size = 50

### DO NOT CHANGE BELOW ###
in_tensor = tf.random.uniform([batch_size, in_height, in_width, in_channels], dtype=tf.dtypes.float16)
filters = tf.random.uniform([filter_height, filter_width, in_channels, out_channels], dtype=tf.dtypes.float16)

# Warming up
%timeit -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)

# Profile!
elapsed_time = %timeit -o -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)
print("batch_size %d: %f GFLOPS" % (batch_size, approximate_gflops(in_tensor, filters, layout, elapsed_time.average)))


## Section 2. Tensor Cores

### Problem 2
We will apply 2D convolution by this settings.

- Layout: NCHW, Batch size = 8
- Input tensor: height = width = 64, channel = 19
- Filter: 3x3
- Output tensor: channel = 131

**Task:** Increase the throughput of the convolution over 11500 GFLOPS by adjusting these values.
- Input channel should be in the range `[12, 30]`.
- Output channel should be in the range `[128, 192]`.
- Tensor layout should be `NHWC` or `NCHW`.

In [ ]:
# Code for Problem 2
import tensorflow as tf
from tensorflow import nn as nn
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# diltion, padding, and stride are not accounted for!
# They are assumed to be default values
def get_conv2d_MNK(in_tensor, filter, layout):
    if layout == "NHWC":
        batch_size, in_height, in_width, in_channels = in_tensor.shape

    if layout == "NCHW":
        batch_size, in_channels, in_height, in_width = in_tensor.shape

    filter_height, filter_width, in_channels, out_channels = filter.shape

    out_height = in_height - (filter_height - 1)
    out_width = in_width - (filter_width - 1)

    M = batch_size * out_height * out_width
    N = out_channels
    K = in_channels * filter_width * filter_height

    return (M, N, K)


def approximate_gflops(in_tensor, filters, layout, time):
    if time <= 0:
        raise ValueError("time must be greater than 0")
    M, N, K = get_conv2d_MNK(in_tensor, filters, layout)
    flop = 2*M*N*K
    flops = flop / time
    gflops = flops / 1000000000
    return gflops


### DO NOT CHANGE THESE VALUES ###
batch_size = 8
in_height = 64
in_width = 64
filter_height = 3
filter_width = 3

### CHANGE HERE! ###
in_channels = 19
out_channels = 131
layout = "NCHW"
in_tensor = tf.random.uniform([batch_size, in_channels, in_height, in_width], dtype=tf.dtypes.float16)
filters = tf.random.uniform([filter_height, filter_width, in_channels, out_channels], dtype=tf.dtypes.float16)

### DO NOT CHANGE BELOW ###
# Warming up
%timeit -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)

# Profile!
elapsed_time = %timeit -o -q nn.conv2d(in_tensor, filters, 1, "VALID", data_format=layout)
print("out_channels %d: %f GFLOPS" % (out_channels, approximate_gflops(in_tensor, filters, layout, elapsed_time.average)))
